# Lab Task - 7
1) Write a program to generate Hash code for a given text using SHA 512
2) Write a program to send a message ensuring confidentiality and integrity

In [21]:
from hashlib import sha512
def hash512(data):
    output = sha512(data.encode()).hexdigest()
    return (output)
data = input('Enter plaintext data: ')
print(hash512(data))

Enter plaintext data: This is going to be hashed
2666bf331ea36a8b4f480880307293e2f65c5490c789b45be8fcd1bdf1644a4c708785d618db445f090efe2f1275663193274906b19a7e6e7f7dea693a7c4ad6


# Server

In [13]:
from Crypto.Cipher import AES 
from Crypto.Util.Padding import pad, unpad
import socket
from cryptography.hazmat.primitives.asymmetric import ec
from cryptography.hazmat.primitives import serialization

def format_key(key):
    return key.ljust(16)[:16].encode()

def encrypt(message, key):
    key = format_key(key)  
    iv = b'1234567890123456'
    cipher = AES.new(key, AES.MODE_CBC, iv)
    return cipher.encrypt(pad(message.encode(), AES.block_size))

def decrypt(ciphertext, key):
    key = format_key(key)
    iv = b'1234567890123456'
    cipher = AES.new(key, AES.MODE_CBC, iv)
    return unpad(cipher.decrypt(ciphertext), AES.block_size).decode()

In [20]:
def server():
    server = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    server.bind(('localhost', 1234))
    server.listen(1)
    print("Server listening on port 1234...")
    
    conn, addr = server.accept()
    print(f"Connection established with {addr}")
    
    server_private_key = ec.generate_private_key(ec.SECP256R1())
    server_public_key = server_private_key.public_key()

    server_public_bytes = server_public_key.public_bytes(
        encoding=serialization.Encoding.PEM,
        format=serialization.PublicFormat.SubjectPublicKeyInfo
    )
    conn.sendall(server_public_bytes)

    client_public_bytes = conn.recv(1024)
    client_public_key = serialization.load_pem_public_key(client_public_bytes)

    shared_secret = server_private_key.exchange(ec.ECDH(), client_public_key)
    print("Server derived shared secret key:", shared_secret.hex())
    
    data = input('Enter plaintext data: ')
    hashcode = hash512(data)
    text = data +'***'+ hashcode
    
    encrypted_text = encrypt(text, shared_secret.hex())
    conn.sendall(encrypted_text)
    
    conn.close()
    server.close()

server()

Server listening on port 12345...
Connection established with ('127.0.0.1', 51004)
Server derived shared secret key: 3c859a3f900095db5248b45c8410ca1bdafbb2144b486f51ccbab4079c465d03
Enter plaintext data: hello client
